In [ ]:
!pip install transformers

In [5]:
import torch
import torch.nn as nn
import random
import tqdm
import torch.optim as optim
import pandas as pd
from nltk.tokenize import TweetTokenizer, word_tokenize
import re
from collections import defaultdict
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import random_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split as tts
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup


In [ ]:
PRE_TRAINED_MODEL_NAME = 'prajjwal1/bert-tiny'
device = 'cuda'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
EPOCHS = 10
MAX_LEN = 512
BATCH_SIZE = 16

In [6]:
train_data = pd.read_csv('../train.csv')
Y = list((train_data['Type'] == 'Quality').astype(int))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('../train.csv')[['Tweet', 'Type']]
df['msg'] = df['Tweet']
df['spam'] = (1 * (df['Type'] == 'Quality'))
df.drop('Tweet')
df.drop('Type')
df.head()

In [ ]:

class SpamDataset(Dataset):
    def __init__(self, spam, msgs, tokenizer, max_len):
        self.msgs = msgs
        self.spam = spam
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.msgs)

    def __getitem__(self, i):
        msg = str(self.msgs[i])
        spam = self.spam[i]

        encoding = self.tokenizer.encode_plus(
            msg,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'msg': msg,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'spam': torch.tensor(spam, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = SpamDataset(
        spam=df['spam'].to_numpy(),
        msgs=df['msg'].to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
    )

df_train, df_test = tts(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True,
)

train_data_loader = create_data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)


In [ ]:
class biLSTM(nn.Module):
    def __init__ (self, h_dim = 10, input_dim = 100, lstm_layers = 1, dropout_rate = 0.2):
        super(biLSTM, self).__init__()
        
        self.h_dim = h_dim
        self.pool = torch.nn.AdaptiveAvgPool1d(output_size=1)

        self.lstm = nn.LSTM(input_size = input_dim, 
                            hidden_size = self.h_dim, 
                            num_layers = lstm_layers,
                            batch_first = True,
                            bidirectional = True)
        
        self.drop = nn.Dropout(p = dropout_rate)
        self.linear = nn.Linear(2 * self.h_dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, X):
        return self.sigmoid(self.linear(self.lstm(X)[0]))

In [ ]:
model = biLSTM(h_dim=100, lstm_layers=2, input_dim=MAX_LEN).to(device)

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [ ]:
def shuffle_data(X, Y):
    indices = list(range(len(X)))
    random.shuffle(indices)
    shuffled_X = X[indices]
    shuffled_Y = Y[indices]
    return (shuffled_X, shuffled_Y)

def train(X, Y, lstm, print_results = False, n_epochs = 50):
    optimizer = optim.Adam(lstm.parameters(), lr = 0.001)
    # optimizer = optim.Adagrad(lstm.parameters(), lr = 0.001)
    # optimizer = optim.SGD(lstm.parameters(), lr = 0.001)
    loss_f = nn.BCELoss()

    train_size = len(X) // 10 * 8
    test_size = len(X) - train_size
    train_ind, test_ind = random_split(range(len(X)), [train_size, test_size], generator=torch.Generator().manual_seed(42))

    batchSize = 10

    train_X = X.clone()[train_ind].to(device = device)
    train_Y = torch.tensor(Y, device = device, dtype = float)[train_ind]
    test_X = X.clone()[test_ind].to(device = device)
    test_Y = torch.tensor(Y, device = device, dtype = float)[test_ind]


    max_f1 = 0
    max_acc = 0
    max_epoch = 0

    for epoch in range(n_epochs):
        lstm.train()
        totalLoss = 0.0

        train_X_shuffle, train_Y_shuffle = shuffle_data(train_X, train_Y)

        for batch in tqdm.tqdm(range(0, len(train_X_shuffle), batchSize), leave=False):
          lstm.zero_grad()
          x = train_X_shuffle[batch: batch + batchSize]
          y = train_Y_shuffle[batch: batch + batchSize]
          output = lstm(x)
          loss = loss_f(output.squeeze().to(dtype = float), y)
          totalLoss += loss.item()
          loss.backward()
          optimizer.step()

        
        lstm.eval()
        output = lstm(test_X).squeeze()
        output = (output > 0.5).int()
        y_pred = output.tolist()

        results = classification_report(test_Y.cpu(), y_pred, labels = [1, 0], digits = 4, output_dict=True)
        if print_results:
          print("============================================")
          print(f"epoch {epoch + 1}")
          print(f"loss: {totalLoss:.4f}")
          print(f"accuracy: {results['accuracy']:.4f}")
          print(f"f1-score: {results['macro avg']['f1-score']:.4f}")
        if results['macro avg']['f1-score'] > max_f1:
          max_f1 = results['macro avg']['f1-score']
          max_acc = results['accuracy']
          max_epoch = epoch + 1
    print("============================================")
    print(f"Best result at epoch {max_epoch} f1-score: {max_f1:.4f} accuracy: {max_acc:.4f}")
        
        


In [ ]:
train(X, Y, lstm, print_results=True, n_epochs=10)

torch.save(lstm, 'model.pt')

  0%|          | 0/957 [00:00<?, ?it/s]

epoch 1
loss: 288.5225
accuracy: 0.9258
f1-score: 0.9258


  0%|          | 0/957 [00:00<?, ?it/s]

epoch 2
loss: 48.9635
accuracy: 0.9179
f1-score: 0.9179


  0%|          | 0/957 [00:00<?, ?it/s]

KeyboardInterrupt: ignored